In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
import lxml
import requests
import unicodedata

In [2]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [3]:
fm_df = pd.read_csv("players_db/fm23/uluta_fm23data.csv")

In [4]:
fm_df.columns = [col.strip() for col in fm_df.columns]

for col in fm_df.select_dtypes("object").columns:
   fm_df[col] = fm_df[col].apply(lambda x: x.strip())

In [5]:
# Assume df is your DataFrame
new_column_names = {
    'Acc': 'Acceleration',
    'Aer': 'Aerial Reach',
    'Agg': 'Aggression',
    'Agi': 'Agility',
    'Ant': 'Anticipation',
    'Bal': 'Balance',
    'Bra': 'Bravery',
    'Cmd': 'Command of Area',
    'Com': 'Communication',
    'Cmp': 'Composure',
    'Cnt': 'Concentration',
    'Cor': 'Corners',
    'Cro': 'Crossing',
    'Dec': 'Decisions',
    'Det': 'Determination',
    'Dri': 'Dribbling',
    'Ecc': 'Eccentricity',
    'Fin': 'Finishing',
    'Fir': 'First Touch',
    'Fla': 'Flair',
    'Fre': 'Free Kick Taking',
    'Han': 'Handling',
    'Hea': 'Heading',
    'Jum': 'Jumping Reach',
    'Kic': 'Kicking',
    'Ldr': 'Leadership',
    'Lon': 'Long Shots',
    'L Th': 'Long Throws',
    'Mar': 'Marking',
    'Nat .1': 'Fitness',
    'OtB': 'Off the Ball',
    '1v1': 'One vs One',
    'Pac': 'Pace',
    'Pas': 'Passing',
    'Pen': 'Penalty Taking',
    'Pos': 'Positioning',
    'Pun': 'Punching',
    'Ref': 'Reflexes',
    'TRO': 'Rushing Out',
    'Sta': 'Stamina',
    'Str': 'Strength',
    'Tck': 'Tackling',
    'Tea': 'Teamwork',
    'Tec': 'Technique',
    'Thr': 'Throwing',
    'Vis': 'Vision',
    'Wor': 'Work Rate'
}

# new_column_names = {key: value.lower() for key, value in new_column_names.items()}
fm_df.rename(columns=new_column_names, inplace=True)

In [6]:
fm_attributes = list(new_column_names.values())
gk_attributes = [ "Command of Area", "Communication", "First Touch", "Handling", "Kicking", "One vs One", "Punching", "Reflexes", "Rushing Out", "Throwing" ]
gk_technical_attributes = [ "Technique" ] # "Free Kick Taking", "Penalty Taking", 
mental_attributes = [ "Aggression", "Anticipation", "Bravery", "Composure", "Concentration", "Decisions", "Determination", "Flair", "Leadership", "Off the Ball", "Positioning", "Teamwork", "Vision", "Work Rate" ]
physical_attributes = [ "Acceleration", "Agility", "Balance", "Jumping Reach", "Fitness", "Pace", "Stamina", "Strength" ]
technical_attributes = [ "Corners", "Crossing", "Dribbling", "Finishing", "First Touch", "Free Kick Taking", "Heading", "Long Shots", "Long Throws", "Marking", "Passing", "Penalty Taking", "Tackling", "Technique" ]

fm_df[fm_attributes] = fm_df[fm_attributes] * 5.5

In [7]:
fm_df['DoB'][0]

'14/8/1984 (37 years old)'

In [8]:
convert_date_format = lambda x: pd.to_datetime(x, format='%d/%m/%Y').strftime('%Y-%m-%d')
fm_df['DoB'] = fm_df['DoB'].apply(lambda x: x.split(" ")[0])
fm_df['DoB'] = fm_df['DoB'].apply(convert_date_format)

In [9]:
fm_df['DoB'][0]

'1984-08-14'

In [10]:
fm_df['Height'] = fm_df['Height'].apply(lambda x: x.split(" ")[0])

In [11]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best Pos', 'Acceleration', 'Aerial Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command of Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First Touch', 'Flair', 'Free Kick Taking', 'Handling', 'Heading',
       'Jumping Reach', 'Kicking', 'Leadership', 'Long Shots', 'Long Throws',
       'Marking', 'Fitness', 'Off the Ball', 'One vs One', 'Pace', 'Passing',
       'Penalty Taking', 'Positioning', 'Punching', 'Reflexes', 'Rushing Out',
       'Stamina', 'Strength', 'Tackling', 'Teamwork', 'Technique', 'Throwing',
       'Vision',

In [12]:
fm_df['Club_id'] = pd.Categorical(fm_df['Club']).codes

In [13]:
fm_df = fm_df[fm_df['Inf'] != 'Yth']

In [14]:
fm_df = fm_df.rename(columns={"Best Pos": "Best_Pos"})

<br><br><br><br>
# Merged_DF

In [15]:
selected_leagues = ['English Premier Division',
 'Ligue 1 Uber Eats',
 'Spanish First Division',
 'Bundesliga',
 'Italian Serie A',
 'Eredivisie',
 'Portuguese Premier League',
 'Turkish Super League']

In [16]:
fm_df = fm_df[fm_df.Division.isin(selected_leagues)]

In [17]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best_Pos', 'Acceleration', 'Aerial Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command of Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First Touch', 'Flair', 'Free Kick Taking', 'Handling', 'Heading',
       'Jumping Reach', 'Kicking', 'Leadership', 'Long Shots', 'Long Throws',
       'Marking', 'Fitness', 'Off the Ball', 'One vs One', 'Pace', 'Passing',
       'Penalty Taking', 'Positioning', 'Punching', 'Reflexes', 'Rushing Out',
       'Stamina', 'Strength', 'Tackling', 'Teamwork', 'Technique', 'Throwing',
       'Vision',

In [18]:
for leg in selected_leagues:
    q = fm_df.query(f"Division == '{leg}'")["Club"].value_counts()
    print(leg , "------", len(q.to_list()))

English Premier Division ------ 20
Ligue 1 Uber Eats ------ 20
Spanish First Division ------ 20
Bundesliga ------ 18
Italian Serie A ------ 20
Eredivisie ------ 18
Portuguese Premier League ------ 18
Turkish Super League ------ 19


In [19]:
fm_df.query(f"Division == 'Turkish Super League'")["Club"].value_counts()

Club
Kayserispor         41
Beşiktaş            41
Trabzonspor         40
Adana D.S.          39
Fenerbahçe          37
Ankaragücü          37
İstanbulspor        36
Galatasaray         35
Hatayspor           35
Kasımpaşa           35
Antalyaspor         34
Alanyaspor          34
Başakşehir FK       34
Sivasspor           33
Giresunspor         33
Gaziantep FK        32
Fatih Karagümrük    32
Ümraniyespor        32
Konyaspor           31
Name: count, dtype: int64

In [20]:
# fm_df.query(f"Club == 'Adana D.S.'").head()

<br><br><br><br>
# Team Names Converter from fminside.com

In [21]:
url_df = fm_df.drop_duplicates(subset=["Club"], keep="first")
url_df.loc[:, 'UID'] = url_df['UID'].apply(lambda text: str(text))
url_df = url_df[["Club", "UID"]]

In [22]:
def Player_to_URL(player_id):
    slug = player_id + "-x-x"
    return f"https://fminside.net/players/3-fm-23/{slug}"

In [23]:
def Convert_Team_Name_From_URL(player_id, club):
    url = Player_to_URL(player_id)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")
    club_name_element = soup.find('div', class_='meta').find('ul').find_all('li')[0].text
    club_name = club_name_element.strip()
    return {"fm":club,"normal":club_name}

In [24]:
team_mapping_array = []
raw_names = url_df.to_dict()
uids = url_df["UID"]
clubs = url_df["Club"]
clubs_array = clubs.to_list()

In [25]:
# for uid,club in zip(uids,clubs):
#     new_entry = Convert_Team_Name_From_URL(uid, club)
#     print(new_entry)
#     team_mapping_array.append(new_entry)

In [26]:
fm_df.query("Name == 'Joshua Kimmich'")

UID Inf            Name         DoB  Nat    Division       Club  \
42332  92039023      Joshua Kimmich  1995-02-08  GER  Bundesliga  FC Bayern   

                      Based Preferred Foot   Right Foot  ... Rushing Out  \
42332  Germany (Bundesliga)          Right  Very Strong  ...        16.5   

      Stamina Strength Tackling  Teamwork Technique Throwing Vision Work Rate  \
42332   110.0     60.5     66.0      93.5      88.0      5.5   99.0     104.5   

      Club_id  
42332    3704  

[1 rows x 75 columns]

In [27]:
fminside_normal_names = pd.read_csv("players_db/fm23/fminside_team_names.csv")
# merged_df = pd.merge(fm_df, fminside_normal_names, how='inner', left_on='Club', right_on='fm')
# merged_df['Club'] = merged_df['normal'].where(merged_df['Club'] == merged_df['fm'], merged_df['Club'])
# # merged_df.drop(['fm', 'normal'], axis=1, inplace=True)
# merged_df = merged_df[merged_df.fm.isin(clubs_array)]

In [28]:
fminside_club_names = []
for index, row in fminside_normal_names.iterrows():
  x_fm = row["fm"]
  x_normal = row["normal"]
  club_dict = {"fm": x_fm, "normal": x_normal}
  fminside_club_names.append(club_dict)

In [29]:
def Rename_Club(club_name):
    matched_club = ""
    for f in fminside_club_names:
        if f["fm"] == club_name:
            return f["normal"]

In [30]:
fm_df['Club'] = fm_df["Club"].apply(lambda x: Rename_Club(x))

In [31]:
fm_df.query("Name == 'Joshua Kimmich'")

UID Inf            Name         DoB  Nat    Division       Club  \
42332  92039023      Joshua Kimmich  1995-02-08  GER  Bundesliga  FC Bayern   

                      Based Preferred Foot   Right Foot  ... Rushing Out  \
42332  Germany (Bundesliga)          Right  Very Strong  ...        16.5   

      Stamina Strength Tackling  Teamwork Technique Throwing Vision Work Rate  \
42332   110.0     60.5     66.0      93.5      88.0      5.5   99.0     104.5   

      Club_id  
42332    3704  

[1 rows x 75 columns]

<br><br><br><br>
# Position Styling

In [32]:
position_mapping = {
    'M (C)': 'MC',
    'D (C)': 'DC',
    'GK': 'GK',
    'ST (C)': 'ST',
    'AM (L)': 'AML',
    'AM (R)': 'AMR',
    'D (R)': 'DR',
    'D (L)': 'DL',
    'DM': 'DM',
    'AM (C)': 'AMC',
    'M (L)': 'ML',
    'M (R)': 'MR',
    'WB (L)': 'WBL',
    'WB (R)': 'WBR'
}

In [33]:
def remove_parentheses(text):
    return text.replace("(", "").replace(")", "")

In [34]:
def pos_map(position):
    return position_mapping[position]

In [35]:
fm_df['Best_Pos'].value_counts()

Best_Pos
M (C)     977
D (C)     938
AM (L)    738
GK        704
ST (C)    648
AM (R)    555
D (R)     521
D (L)     475
DM        388
AM (C)    180
M (L)     130
M (R)     116
WB (R)     29
WB (L)     23
Name: count, dtype: int64

In [36]:
fm_df['Position'].value_counts()

Position
D (C)                    749
GK                       704
DM, M (C)                600
ST (C)                   519
M/AM (C)                 279
                        ... 
D (R), WB/M (RL)           1
D (R), WB (RL), M (R)      1
D (RLC), WB/M (R)          1
WB/M (RL)                  1
D/AM (R)                   1
Name: count, Length: 224, dtype: int64

In [37]:
def Multiple_Position_Beautifier(position_value):
    x = ""
    positions = position_value.split(",")
    for p in positions:
        p = p.strip()
        if p in position_mapping.keys():
            x += f"{position_mapping[p]},"
        elif "/" in p:
            slashed = p.split(" ")
            locations = slashed[0].split("/")
            sides = list(remove_parentheses(slashed[1]))
            for loc in locations:
                for side in sides:
                    splitted_pos = loc+side
                    x += f"{splitted_pos},"
        else:
            slashed = p.split(" ")
            loc = slashed[0]
            sides = list(remove_parentheses(slashed[1]))
            for side in sides:
                splitted_pos = loc+side
                x += f"{splitted_pos},"
    return x[:-1]

In [38]:
fm_df['Position'] = fm_df['Position'].apply(lambda row: Multiple_Position_Beautifier(row))
fm_df['Best_Pos'] = fm_df['Best_Pos'].apply(lambda row: Multiple_Position_Beautifier(row))

In [39]:
position_order = ['GK', 'WBR', 'DR', 'DC', 'DL', 'WBL', 'DM',  
                  'MC', 'MR', 'AMC', 'ML', 'AMR', 'ST', 'AML']
def get_position_rank(position):
    return position_order.index(position) if position in position_order else len(position_order)
fm_df['Pos_Rank'] = fm_df['Best_Pos'].apply(get_position_rank)

In [40]:
fm_df.query("Club == 'Chaves'")[["Best_Pos"]]

Best_Pos
168395       DC
168409       DC
168573      AML
168584       ST
168616      AML
168825      AML
169038      AMR
169074       GK
169161       MC
169202       ML
169207      AMC
169215       DR
169352       ST
169530      AML
169540      AML
169732       MC
169971       DM
170215       DL
170250      AMR
170287      AMR
170296      WBR
170327       ST
171625       MC
173209       GK
173950      AMR
177476      AMR
178013       DC
183728       DC

In [41]:
fm_df['Position'].value_counts()

Position
DC                  749
GK                  704
DM,MC               600
ST                  519
MC,AMC              279
                   ... 
DR,WBR,WBL,MR,ML      1
DR,WBR,WBL,MR         1
DR,DL,DC,WBR,MR       1
WBR,WBL,MR,ML         1
DR,AMR                1
Name: count, Length: 224, dtype: int64

In [42]:
fm_df['Best_Pos'].value_counts()

Best_Pos
MC     977
DC     938
AML    738
GK     704
ST     648
AMR    555
DR     521
DL     475
DM     388
AMC    180
ML     130
MR     116
WBR     29
WBL     23
Name: count, dtype: int64

<br><br><br><br>
# Players' Overall

- Keeper -> gk
- Defending -> def 
- Passing -> pas
- Dribbling -> dri
- Finishing -> fin
- Stamina -> sta
- Strength -> str
- Heading -> hed
- Mental -> men
- Intelligence -> iq

In [43]:
rating_attrs = ["gk","def","pas","dri","fin","sta","str","hed","men","iq"]

In [44]:
def Dict_Generator(attrs):
    weights = (len(attrs)*"0.1,").split(",")[:-1]
    return {key: float(weights[i]) for i, key in enumerate(attrs)}

<h3 style="color:orange;">  Keeper</h3>

In [45]:
current_ability = "gk"
fm_df[current_ability] = fm_df[gk_attributes + gk_technical_attributes].mean(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  gk
167966  Marc-André ter Stegen  83
42335            Manuel Neuer  82
151647                Alisson  82
151645                Ederson  81
167963              Jan Oblak  80

<h3 style="color:orange;">  Defending</h3>

In [46]:
mental_attributes = [ "Aggression", "Anticipation", "Bravery", "Composure", "Concentration", "Decisions", "Determination", "Flair", "Leadership", "Off the Ball", "Positioning", "Teamwork", "Vision", "Work Rate" ]
physical_attributes = [ "Acceleration", "Agility", "Balance", "Jumping Reach", "Fitness", "Pace", "Stamina", "Strength" ]
technical_attributes = [ "Corners", "Crossing", "Dribbling", "Finishing", "First Touch", "Free Kick Taking", "Heading", "Long Shots", "Long Throws", "Marking", "Passing", "Penalty Taking", "Tackling", "Technique" ]

In [47]:
expert_attributes = ["Anticipation", "Bravery", "Acceleration", "Concentration", "Decisions", "Positioning", "Jumping Reach", "Fitness", "Pace", "Strength", "Heading", "Marking", "Tackling" ]

weights =   {'Anticipation': 0.05,
             'Bravery': 0.05,
             "Acceleration": 0.05,
             'Concentration': 0.05,
             'Decisions': 0.05,
             'Positioning': 0.05,
             'Jumping Reach': 0.05,
             'Fitness': 0.1,
             'Pace': 0.05,
             'Strength': 0.15,
             'Heading': 0.15,
             'Marking': 0.1,
             'Tackling': 0.1}.values()

current_ability = "def"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  def
151644  Virgil van Dijk   92
73546    Milan Škriniar   90
151642       Rúben Dias   89
73611    Chris Smalling   87
73641       Kim Min-Jae   87

<h3 style="color:red;">  Passing</h3>

In [48]:
expert_attributes = ["Crossing", "First Touch", "Passing", "Technique", "Decisions", "Determination", "Vision"]
weights =   {'Crossing': 0.10,
             'First Touch': 0.10,
             'Passing': 0.35,
             'Technique': 0.20,
             'Decisions': 0.05,
             'Determination': 0.05,
             'Vision': 0.15}

current_ability = "pas"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  pas
167956     Lionel Messi  104
167959      Luka Modrić  102
151638  Kevin De Bruyne   99
167960    Karim Benzema   99
167970       Toni Kroos   98

<h3 style="color:red;">  Dribbling</h3>

In [49]:
expert_attributes = ["Dribbling","First Touch","Acceleration","Agility","Balance","Fitness","Pace"]

weights =   {'Dribbling': 0.35,
             'First Touch': 0.05,
             'Acceleration': 0.20,
             'Agility': 0.10,
             'Balance': 0.10,
             'Fitness': 0.05,
             'Pace': 0.15}

current_ability = "dri"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  dri
167957    Kylian Mbappé   99
151738     Adama Traoré   96
151640    Mohamed Salah   95
167956     Lionel Messi   94
167973  Ousmane Dembélé   94

<h3 style="color:red;">  Finishing</h3>

In [50]:
expert_attributes = ["Finishing"]
weights =   {'Finishing': 1}.values()

current_ability = "fin"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  fin
151646   Cristiano Ronaldo  104
73550   Zlatan Ibrahimović  104
167958  Robert Lewandowski  104
151641          Harry Kane  104
73547        Ciro Immobile   99

<h3 style="color:green;">  Stamina</h3>

In [51]:
expert_attributes = ["Stamina"]
weights =   {'Stamina': 1}.values()

current_ability = "sta"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  sta
42332          Joshua Kimmich  110
151729      James Ward-Prowse  110
73553        Marcelo Brozović  110
151691            Mason Mount  110
151758  Pierre-Emile Højbjerg  110

<h3 style="color:green;">  Strength</h3>

In [52]:
expert_attributes = ["Strength"]
weights =   {'Strength': 1}.values()

current_ability = "str"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  str
151713           Kurt Zouma  110
73542         Romelu Lukaku  110
152008       Fraser Forster  104
151685          Reece James  104
151812  Aleksandar Mitrović  104

<h3 style="color:green;">  Heading</h3>

In [53]:
expert_attributes = ["Heading"]
weights =   {'Heading': 1}.values()

current_ability = "hed"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  hed
167965       Sergio Ramos  110
73561      Olivier Giroud  104
75806         Milan Đurić  104
151646  Cristiano Ronaldo  104
167994     Edinson Cavani   99

<h3 style="color:purple;">  Mental</h3>

In [54]:
expert_attributes = ["Aggression", "Composure", "Concentration"]
weights =   {'Aggression': 0.15,
             'Composure': 0.40,
             'Concentration': 0.45}

current_ability = "men"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  men
151660     Thiago Silva  100
167993      Axel Witsel   95
167980  Sergio Busquets   94
167959      Luka Modrić   94
151645          Ederson   93

<h3 style="color:purple;">  Intelligence</h3>

In [55]:
expert_attributes = ["Vision", "Positioning", "Anticipation", "Decisions", "Flair"]
weights =   {'Vision': 0.35,
             'Positioning': 0.15,
             'Anticipation': 0.20,
             'Decisions': 0.15,
             'Flair': 0.15}

current_ability = "iq"
fm_df[current_ability] = (fm_df[expert_attributes] * weights).sum(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

Name  iq
167959      Luka Modrić  93
167956     Lionel Messi  92
167988   Marco Verratti  91
151638  Kevin De Bruyne  91
151656           Thiago  90

<h1 style="color:red;">  🥽TPR</h1>

In [56]:
positions = fm_df['Best_Pos'].unique().tolist()

In [57]:
def TPR_by_Position(row, pos=False):
    if not pos:
        position = row["Best_Pos"]
    else:
        position = pos
        
    rating_weights = {}
    if position == "GK":
        rating_weights = { "gk": 0.95, "def": 0, "pas": 0, "dri": 0, "fin": 0, "sta": 0, "str": 0, "hed": 0, "men": 0.05, "iq": 0 }
    elif position == "DC":
        rating_weights = { "gk": 0, "def": 0.25, "pas": 0.12, "dri": 0, "fin": 0, "sta": 0.19, "str": 0.15, "hed": 0.15, "men": 0.1, "iq": 0.05 }
    elif position == "DL" or position == "DR":
        rating_weights = { "gk": 0, "def": 0.22, "pas": 0.25, "dri": 0.23, "fin": 0, "sta": 0.17, "str": 0.04, "hed": 0.02, "men": 0.03, "iq": 0.04 }
    elif position == "WBL" or position == "WBR":
        rating_weights = { "gk": 0, "def": 0.09, "pas": 0.21, "dri": 0.32, "fin": 0, "sta": 0.22, "str": 0.04, "hed": 0.01, "men": 0.03, "iq": 0.08 }
    elif position == "DM":
        rating_weights = { "gk": 0, "def": 0.1, "pas": 0.23, "dri": 0.09, "fin": 0.02, "sta": 0.19, "str": 0.1, "hed": 0.04, "men": 0.08, "iq": 0.15 }
    elif position == "MC":
        rating_weights = { "gk": 0, "def": 0.01, "pas": 0.27, "dri": 0.15, "fin": 0.13, "sta": 0.11, "str": 0.07, "hed": 0.02, "men": 0.04, "iq": 0.2 }
    elif position == "ML" or position == "MR":
        rating_weights = { "gk": 0, "def": 0.07, "pas": 0.11, "dri": 0.2, "fin": 0.14, "sta": 0.17, "str": 0.11, "hed": 0.03, "men": 0.07, "iq": 0.09 }
    elif position == "AMC":
        rating_weights = { "gk": 0, "def": 0, "pas": 0.25, "dri": 0.19, "fin": 0.16, "sta": 0.06, "str": 0.04, "hed": 0, "men": 0.03, "iq": 0.27 }
    elif position == "AML" or position == "AMR":
        rating_weights = { "gk": 0, "def": 0, "pas": 0.08, "dri": 0.34, "fin": 0.22, "sta": 0.16, "str": 0.03, "hed": 0.01, "men": 0.05, "iq": 0.11 }
    elif position == "ST":
        rating_weights = { "gk": 0, "def": 0, "pas": 0.09, "dri": 0.14, "fin": 0.3, "sta": 0.12, "str": 0.1, "hed": 0.12, "men": 0.06, "iq": 0.07 }
    
    rating_attrs = list(rating_weights.keys())
    tpr = (row[rating_attrs] * pd.Series(rating_weights)).sum()
    return int(tpr)

In [63]:
fm_df["tpr"] = fm_df.apply(TPR_by_Position, axis=1)

for pos in positions:
    fm_df[f"tpr_{pos}"] = 0

In [70]:
posdf = pd.read_csv("position_regression_formulas.csv")

In [71]:
def Position_Regression(row, pos):
    best_pos = row.Best_Pos
    x = row.tpr
    m = posdf[posdf.Best_Pos == best_pos][f"{pos}_slope"]
    b = posdf[posdf.Best_Pos == best_pos][f"{pos}_intercept"]
    y = int(m*x + b)
    return max(y, 0) 

In [73]:
for pos in positions:
    fm_df[f"tpr_{pos}"] = fm_df.apply(lambda row: TPR_by_Position(row, pos) if pos == row.Best_Pos or pos in row.Position else Position_Regression(row, pos), axis=1)

In [74]:
fm_df.sort_values("tpr",ascending=False)[["Name","tpr"]].head(n=10)

Name  tpr
167958  Robert Lewandowski   92
167956        Lionel Messi   91
151644     Virgil van Dijk   89
151641          Harry Kane   89
151640       Mohamed Salah   88
151638     Kevin De Bruyne   88
151642          Rúben Dias   87
167957       Kylian Mbappé   87
151639      Erling Haaland   87
73542        Romelu Lukaku   86

In [75]:
fm_df.query("Name in 'Lionel Messi'").iloc[0].to_dict()

{'UID': 7458500,
 'Inf': '',
 'Name': 'Lionel Messi',
 'DoB': '1987-06-24',
 'Nat': 'ARG',
 'Division': 'Ligue 1 Uber Eats',
 'Club': 'Paris Saint-Germain',
 'Based': 'France (Ligue 1 Uber Eats)',
 'Preferred Foot': 'Left',
 'Right Foot': 'Fairly Strong',
 'Left Foot': 'Very Strong',
 'Position': 'AMR,AMC,ST',
 'Height': '169',
 'Weight': '67 kg',
 'Age': 35,
 'Transfer Value': '€50M - €76M',
 'Wage': '€3,366,000 p/m',
 'AT Apps': '578',
 'AT Gls': '491',
 'Team': 'Main',
 'Caps': 162,
 'Yth Apps': '18',
 'Style': 'Creative',
 'Rc Injury': '-',
 'Best Role': 'Attacking Midfielder',
 'Best Duty': 'Attack',
 'Best_Pos': 'AMC',
 'Acceleration': 82.5,
 'Aerial Reach': 11.0,
 'Aggression': 38.5,
 'Agility': 82.5,
 'Anticipation': 93.5,
 'Balance': 99.0,
 'Bravery': 55.0,
 'Command of Area': 11.0,
 'Communication': 5.5,
 'Composure': 88.0,
 'Concentration': 60.5,
 'Corners': 82.5,
 'Crossing': 82.5,
 'Decisions': 99.0,
 'Determination': 110.0,
 'Dribbling': 110.0,
 'Eccentricity': 5.5,
 'Fin

# CSV EXPORT

In [77]:
fm_df.to_csv("players_db/fm23/fm23db_processed.csv")